In [8]:
!pip install streamlit pyngrok openai requests

In [9]:
!pip install streamlit pyngrok streamlit-folium folium geopy requests -q

In [10]:
!pip install streamlit-folium folium


In [17]:
# Simpan file app.py
with open("app.py", "w") as f:
    f.write('''
import streamlit as st
import requests
import folium
from streamlit_folium import st_folium
from geopy.geocoders import Nominatim

st.set_page_config(page_title="Dashboard Cuaca & Tanah", layout="centered")
st.title("🌦️ Cuaca & 🧪 Tanah Berdasarkan Lokasi")

API_KEY = "d4179fb703532ad460882dd59234d867"  # Ganti dengan API key kamu

# Fungsi ambil koordinat dari kota
def get_coords_from_city(city_name):
    geolocator = Nominatim(user_agent="soil_data_colab")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    return None, None

# Fungsi cuaca dari OpenWeather
def get_forecast_summary(lat, lon, api_key):
    url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_key}&units=metric"
    response = requests.get(url)
    if response.status_code != 200:
        return None
    data = response.json()
    temps, humidities, rainfalls = [], [], []
    for entry in data["list"]:
        temps.append(entry['main']['temp'])
        humidities.append(entry['main']['humidity'])
        rain = entry.get('rain', {}).get('3h', 0)
        rainfalls.append(rain)
    return {
        "Rata-rata Suhu (°C)": round(sum(temps) / len(temps), 2),
        "Rata-rata Kelembapan (%)": round(sum(humidities) / len(humidities), 2),
        "Rata-rata Curah Hujan (mm/3 jam)": round(sum(rainfalls) / len(rainfalls), 2)
    }

# Fungsi SoilGrids
def get_soil_data(lat, lon):
    base_url = "https://rest.isric.org/soilgrids/v2.0/properties/query"
    def get_value(prop):
        params = {
            "property": prop,
            "depth": "0-5cm",
            "value": "mean",
            "lat": lat,
            "lon": lon
        }
        r = requests.get(base_url, params=params)
        if r.status_code == 200:
            try:
                val = r.json()["properties"]["layers"][0]["depths"][0]["values"]["mean"]
                return val / 10.0 if prop == "phh2o" else val
            except:
                return None
        return None
    return {
        "pH Tanah (0–5cm)": get_value("phh2o"),
        "Nitrogen (cg/kg)": get_value("nitrogen")
    }

# Input kota
city = st.text_input("Masukkan nama kota (Contoh: Bandung)")

lat, lon = None, None
if city:
    lat, lon = get_coords_from_city(city)
    if lat:
        st.success(f"Koordinat: ({lat:.2f}, {lon:.2f})")

# Peta interaktif
st.divider()
st.subheader("🗺️ Klik lokasi di peta (opsional)")
m = folium.Map(location=[-2.5, 118], zoom_start=5)
folium.LatLngPopup().add_to(m)
map_data = st_folium(m, width=700, height=500)

if map_data and map_data.get("last_clicked"):
    lat = map_data["last_clicked"]["lat"]
    lon = map_data["last_clicked"]["lng"]
    st.success(f"Koordinat dipilih: ({lat:.2f}, {lon:.2f})")

if lat and lon:
    # Tampilkan ringkasan cuaca
    st.subheader("🌦️ Ringkasan Cuaca")
    cuaca = get_forecast_summary(lat, lon, API_KEY)
    if cuaca:
        for k, v in cuaca.items():
            st.write(f"✅ **{k}:** {v}")
    else:
        st.error("Gagal mendapatkan data cuaca.")

    # Tampilkan data tanah
    st.subheader("🧪 Data Tanah (SoilGrids)")
    tanah = get_soil_data(lat, lon)
    for k, v in tanah.items():
        if v is not None:
            st.write(f"✅ **{k}:** {v}")
        else:
            st.warning(f"❌ {k} tidak tersedia.")
''')

In [15]:
from pyngrok import conf, ngrok

# Masukkan token ngrok kamu di sini
ngrok.set_auth_token("2xH5SFTP0InDNxn9Xe9IHmD49q8_3avvKtEwLivdV5NvZJzfQ")


In [16]:
!streamlit run app.py &>/content/log.txt &

# Hubungkan ke port 8501
public_url = ngrok.connect(addr="http://localhost:8501")
print("Akses aplikasi Anda di:", public_url)


Akses aplikasi Anda di: NgrokTunnel: "https://45a4-34-10-191-54.ngrok-free.app" -> "http://localhost:8501"
